In [1]:
import glob
import pprint
import pickle
import numpy as np
import japanize_matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import xml.etree.ElementTree as et

from Comicle_Class import Comic, MainComic, Character, FeaturedCharacter

In [2]:
ComicInstances = []

for xFile in glob.glob('annotations_Main/*.xml'):
    tree = et.ElementTree(file = xFile)
    root = tree.getroot()
    xTitle = root.attrib["title"]
    xAuthor = root.attrib["author"]
    xEra = root.attrib["era"]
    xPublisher = root.attrib["publisher"]
    xTarget = root.attrib["target"]
    xGenre = root.attrib["genre"]
    xType = root.attrib["type"]

    xComicInstance = MainComic(xFile, xTitle, xAuthor, xEra, xPublisher, xTarget, xGenre, xType, None, None, None, None, None, None, None)
    ComicInstances.append(xComicInstance)

In [3]:
for xComicInstance in ComicInstances:
    xFile = xComicInstance.get_File()
    tree = et.ElementTree(file = xFile)
    root = tree.getroot()
    
    xFrameRanges = []
    for child in root:
        if child.tag == "pages":
            for grandchild in child:
                xFrameRange = 0
                for great_grandchild in grandchild:
                    if great_grandchild.tag == "frame":
                            xmin = int(great_grandchild.attrib["xmin"])
                            ymin = int(great_grandchild.attrib["ymin"])
                            xmax = int(great_grandchild.attrib["xmax"])
                            ymax = int(great_grandchild.attrib["ymax"])
                            xFrameRange += (xmax - xmin) * (ymax - ymin)
                xFrameRanges.append(xFrameRange)
    
    xComicInstance.set_PageCount(len(xFrameRanges))
    xComicInstance.set_FrameRanges(xFrameRanges)

In [4]:
for xComicInstance in ComicInstances:
    xFile = xComicInstance.get_File()
    tree = et.ElementTree(file = xFile)
    root = tree.getroot()

    xCharacterInstances = []
    for child in root:
        if child.tag == "characters":
            for grandchild in child:
                yId = grandchild.attrib["id"]
                yName =  grandchild.attrib["name"]
                yCharacterInstance = Character(yId, yName, None)
                xCharacterInstances.append(yCharacterInstance)
    
    xComicInstance.set_CharacterInstances(xCharacterInstances)                

In [5]:
"""for xComicInstance in ComicInstances:
    xCharacterInstances = xComicInstance.get_CharacterInstances()
    for yCharacterInstance in xCharacterInstances:
        yName = yCharacterInstance.get_Name()
        print(yName)
    print()"""

'for xComicInstance in ComicInstances:\n    xCharacterInstances = xComicInstance.get_CharacterInstances()\n    for yCharacterInstance in xCharacterInstances:\n        yName = yCharacterInstance.get_Name()\n        print(yName)\n    print()'

In [6]:
for xComicInstance in ComicInstances:
    xFile = xComicInstance.get_File()
    tree = et.ElementTree(file = xFile)
    root = tree.getroot()

    SmoothingNumber = 20
    W = np.ones(SmoothingNumber)/SmoothingNumber

    xFrameRanges = xComicInstance.get_FrameRanges()
    xCharacterInstances = xComicInstance.get_CharacterInstances()
    for i in range(len(xCharacterInstances)):
        yCharacterInstance = xCharacterInstances[i]
        yId = yCharacterInstance.get_Id()
        yCharacterRates = []

        PageCount = 0
        for child in root:
            if child.tag == "pages":
                for grandchild in child:
                    yCharacterRange = 0
                    for great_grandchild in grandchild:
                        if great_grandchild.tag == "body":
                            if great_grandchild.attrib["character"] == yId:
                                xmin = int(great_grandchild.attrib["xmin"])
                                ymin = int(great_grandchild.attrib["ymin"])
                                xmax = int(great_grandchild.attrib["xmax"])
                                ymax = int(great_grandchild.attrib["ymax"])
                                yCharacterRange += (xmax - xmin) * (ymax - ymin)
                    if xFrameRanges[PageCount] != 0:
                        yCharacterRate = yCharacterRange / xFrameRanges[PageCount]
                    else:
                        yCharacterRate = 0
                    yCharacterRates.append(yCharacterRate)
                    PageCount += 1
        
        yCharacterRates = np.array(yCharacterRates)
        yCharacterRates = (np.convolve(yCharacterRates, W, mode = "same"))
                
        yCharacterInstance.set_CharacterRates(yCharacterRates)

    xComicInstance.set_CharacterInstances(xCharacterInstances) 

In [7]:
"""for xComicInstance in ComicInstances:
    xTitle = xComicInstance.get_Title()
    if xTitle == "東京トイボクシーズ":
        plt.figure(figsize=(10,4))
        xCharacterInstances = xComicInstance.get_CharacterInstances()
        for yCharacterInstance in xCharacterInstances:
            yCharacterRates = yCharacterInstance.get_CharacterRates()
            plt.plot(yCharacterRates)
        plt.show()"""

'for xComicInstance in ComicInstances:\n    xTitle = xComicInstance.get_Title()\n    if xTitle == "東京トイボクシーズ":\n        plt.figure(figsize=(10,4))\n        xCharacterInstances = xComicInstance.get_CharacterInstances()\n        for yCharacterInstance in xCharacterInstances:\n            yCharacterRates = yCharacterInstance.get_CharacterRates()\n            plt.plot(yCharacterRates)\n        plt.show()'

In [8]:
def make_BorderRate(iPageCharacterRates):
    iPageCharacterRates = sorted(iPageCharacterRates, reverse = True)
    iPageCharacterRateDifferences = []

    for j in range(len(iPageCharacterRates) - 1):
        iPageCharacterRateDifference = abs(iPageCharacterRates[j] - iPageCharacterRates[j + 1])
        iPageCharacterRateDifferences.append(iPageCharacterRateDifference)
    index_iPageMaxRateDifference = iPageCharacterRateDifferences.index(max(iPageCharacterRateDifferences))
    iPageBorderRate = iPageCharacterRates[index_iPageMaxRateDifference]
    
    return iPageBorderRate


for xComicInstance in ComicInstances:
    arrayCharacterRates = []
    xCharacterInstances = xComicInstance.get_CharacterInstances()
    for yCharacterInstance in xCharacterInstances:
        yCharacterRates = yCharacterInstance.get_CharacterRates()
        arrayCharacterRates.append(yCharacterRates)

    #pprint.pprint(np.array(arrayCharacterRates).shape)
    
    xBorderRates = []
    for i in range(len(arrayCharacterRates[0])):
        iPageCharacterRates = np.array(arrayCharacterRates)[:, i].tolist()
        iPageBorderRate = make_BorderRate(iPageCharacterRates)
        xBorderRates.append(iPageBorderRate)
    
    xComicInstance.set_BorderRates(xBorderRates)

In [9]:
for xComicInstance in ComicInstances:
    xPageCount = xComicInstance.get_PageCount()
    xBorderRates = xComicInstance.get_BorderRates()
    xCharacterInstances = xComicInstance.get_CharacterInstances()

    xFeaturedCharacterInstances = []

    for yCharacterInstance in xCharacterInstances:
        yCharacterRates = yCharacterInstance.get_CharacterRates()
        
        yFeaturedCharacterRates = []
        for i in range(len(xBorderRates)):
            iPageyCharacterRate = yCharacterRates[i]
            iPageBorderRate = xBorderRates[i]
            if iPageyCharacterRate >= iPageBorderRate:
                yFeaturedCharacterRates.append(iPageyCharacterRate)
            else:
                yFeaturedCharacterRates.append(np.nan)
        
        if yFeaturedCharacterRates != [np.nan] * xPageCount:
            yId = yCharacterInstance.get_Id()
            yName = yCharacterInstance.get_Name()
            yCharacterRates = yCharacterInstance.get_CharacterRates()
            yFeaturedCharacterInstance = FeaturedCharacter(yId, yName, yCharacterRates, yFeaturedCharacterRates)
            xFeaturedCharacterInstances.append(yFeaturedCharacterInstance)


    yNotFeaturePageCount_yFeaturedCharacterInstance = {} 
    for yFeaturedCharacterInstance in xFeaturedCharacterInstances:
        yFeaturedCharacterRates = yFeaturedCharacterInstance.get_FeaturedCharacterRates()
        yNotFeaturePageCount = yFeaturedCharacterRates.count(np.nan)
        yNotFeaturePageCount_yFeaturedCharacterInstance[yNotFeaturePageCount] = yFeaturedCharacterInstance

    yFeatureRank_yFeaturedCharacterInstance = {}
    ascendingyNotFeaturePageCounts = sorted(list(yNotFeaturePageCount_yFeaturedCharacterInstance.keys()))
    yFeatureRank = 1
    for yNotFeaturePageCount in ascendingyNotFeaturePageCounts:
        yFeaturedCharacterInstance = yNotFeaturePageCount_yFeaturedCharacterInstance[yNotFeaturePageCount]
        yFeatureRank_yFeaturedCharacterInstance[yFeatureRank] = yFeaturedCharacterInstance
        yFeatureRank += 1
    
    xComicInstance.set_FeaturedCharacterCount(len(yFeatureRank_yFeaturedCharacterInstance))
    xComicInstance.set_FeatureRank_FeaturedCharacterInstance(yFeatureRank_yFeaturedCharacterInstance)


In [10]:
MaxCount = 0
for xComicInstance in ComicInstances:
    yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()
    if MaxCount < len(yFeatureRank_yFeaturedCharacterInstance):
        MaxCount = len(yFeatureRank_yFeaturedCharacterInstance)

for xComicInstance in ComicInstances:
    xPageCount = xComicInstance.get_PageCount()
    yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()
    while len(yFeatureRank_yFeaturedCharacterInstance) < MaxCount:
        yFeatureRank_yFeaturedCharacterInstance[len(yFeatureRank_yFeaturedCharacterInstance) + 1] = FeaturedCharacter(None, "架空キャラ", [0] * PageCount, None)

In [11]:
"""for xComicInstance in ComicInstances:
    xTitle = xComicInstance.get_Title()
    if xTitle == "東京トイボクシーズ":
        yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()
        for yFeatureRank in yFeatureRank_yFeaturedCharacterInstance:
            yFeaturedCharacterInstance = yFeatureRank_yFeaturedCharacterInstance[yFeatureRank]
            yName = yFeaturedCharacterInstance.get_Name()
            print(yName, yFeatureRank)
            print()"""

'for xComicInstance in ComicInstances:\n    xTitle = xComicInstance.get_Title()\n    if xTitle == "東京トイボクシーズ":\n        yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()\n        for yFeatureRank in yFeatureRank_yFeaturedCharacterInstance:\n            yFeaturedCharacterInstance = yFeatureRank_yFeaturedCharacterInstance[yFeatureRank]\n            yName = yFeaturedCharacterInstance.get_Name()\n            print(yName, yFeatureRank)\n            print()'

In [12]:
"""Colors = mcolors.TABLEAU_COLORS
Colors = list(Colors.keys())
print(len(Colors))

for xComicInstance in ComicInstances:
    xTitle = xComicInstance.get_Title()
    plt.figure(figsize=(10,4))
    yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()
    yFeatureRank = 1
    while yFeatureRank in yFeatureRank_yFeaturedCharacterInstance:
        yFeaturedCharacterInstance = yFeatureRank_yFeaturedCharacterInstance[yFeatureRank]
        yName = yFeaturedCharacterInstance.get_Name()
        if yName != "架空キャラ":
            yCharacterRates = yFeaturedCharacterInstance.get_CharacterRates()
            yFeaturedCharacterRates = yFeaturedCharacterInstance.get_FeaturedCharacterRates()

            plt.plot(yCharacterRates, color = Colors[yFeatureRank - 1], linewidth = 2, label = yName)
            plt.plot(yFeaturedCharacterRates, color = Colors[yFeatureRank - 1], linewidth = 6)
        
        yFeatureRank += 1

        if yFeatureRank == 11:
            break

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=5)
    plt.tight_layout()
    plt.savefig("Graph/" + xTitle + ".png")
    #plt.show()"""

'Colors = mcolors.TABLEAU_COLORS\nColors = list(Colors.keys())\nprint(len(Colors))\n\nfor xComicInstance in ComicInstances:\n    xTitle = xComicInstance.get_Title()\n    plt.figure(figsize=(10,4))\n    yFeatureRank_yFeaturedCharacterInstance = xComicInstance.get_FeatureRank_FeaturedCharacterInstance()\n    yFeatureRank = 1\n    while yFeatureRank in yFeatureRank_yFeaturedCharacterInstance:\n        yFeaturedCharacterInstance = yFeatureRank_yFeaturedCharacterInstance[yFeatureRank]\n        yName = yFeaturedCharacterInstance.get_Name()\n        if yName != "架空キャラ":\n            yCharacterRates = yFeaturedCharacterInstance.get_CharacterRates()\n            yFeaturedCharacterRates = yFeaturedCharacterInstance.get_FeaturedCharacterRates()\n\n            plt.plot(yCharacterRates, color = Colors[yFeatureRank - 1], linewidth = 2, label = yName)\n            plt.plot(yFeaturedCharacterRates, color = Colors[yFeatureRank - 1], linewidth = 6)\n        \n        yFeatureRank += 1\n\n        if yFea

In [13]:
with open('ComicInstances.bin', 'wb') as p:
    pickle.dump(ComicInstances, p)